<a href="https://colab.research.google.com/github/AdarshShah/final-year-project/blob/master/sepsis_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import math
import pandas as pd
import psycopg2
import getpass
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Model, Sequential
from keras.layers import Input, GRU, Dense, LSTM, Flatten, Reshape, Lambda, CuDNNLSTM, Dropout
import matplotlib.pyplot as plt

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
path = '/content/drive/My Drive/sepsis_dataset.csv'
#path = '/home/adarsh/Documents/sepsis/sepsis_dataset.csv'
sepsis_dataset = pd.read_csv(path)

In [0]:
sepsis_dataset.iloc[:,1:-2] = StandardScaler().fit_transform(sepsis_dataset.iloc[:,1:-2])

In [0]:
train = sepsis_dataset['hadm_id'].drop_duplicates()
x_train, x_test = train_test_split(train,test_size=0.2)

In [0]:
model = Sequential()
model.add(LSTM(units=128,batch_input_shape=(1,None,30),return_sequences=True))
model.add(LSTM(units=64,return_sequences=False))
model.add(Dense(units=64,activation='relu'))
model.add(Dense(units=16,activation='relu'))
model.add(Dense(units=4,activation='relu'))
model.add(Dense(units=1,activation='sigmoid'))

model.compile(optimizer='sgd',loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

In [0]:
i = 1
count = len(x_train)
for x in x_train[0:50]:
  X = np.array(sepsis_dataset[sepsis_dataset['hadm_id']==x].iloc[:,1:-2].drop_duplicates())
  (a,b) = np.shape(X)
  X = np.reshape(X,(1,a,30))
  Y = sepsis_dataset[sepsis_dataset['hadm_id']==x]['sepsis'].drop_duplicates()
  model.fit(X,Y,epochs=10)
  print('\n\n%d/%d\n\n'%(i,count))
  i=i+1

In [0]:
for x in x_train[0:10]:
  X = np.array(sepsis_dataset[sepsis_dataset['hadm_id']==x].iloc[:,1:-2].drop_duplicates())
  (a,b) = np.shape(X)
  X = np.reshape(X,(1,a,30))
  Y = sepsis_dataset[sepsis_dataset['hadm_id']==x]['sepsis'].drop_duplicates()
  print('%d %f'%(Y,model.predict(X)))
  i=i+1